In [ ]:
pip install hvplot

In [ ]:
import requests
from IPython.display import JSON
import pandas as pd
import numpy as np
import time

# Visualization
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px

# Panel/hvplot (holoviz)
import panel as pn
pn.extension()
import param
import hvplot.pandas

# Others
import pickle
from io import StringIO

In [ ]:
API_KEY = '97d3613ff3ff4bae82a83323cb17c66b'

In [ ]:
# Submitting Files for Transcription
import requests
endpoint = "https://api.assemblyai.com/v2/transcript"
json = {
    "audio_url": "https://github.com/niveditavyas/audio_analysis/blob/main/Greek%20Gods%20Explained%20In%2012%20Minutes.mp3?raw=true",
    "auto_highlights": True,
    "sentiment_analysis": True,
    "auto_chapters": True,
    "iab_categories": True,
}
headers = {
    "authorization": API_KEY,
    "content-type": "application/json"
}
response = requests.post(endpoint, json=json, headers=headers)
print(response.json())

{'id': '6mrstfopff-9272-4b53-801a-f26901bff9a5', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://github.com/niveditavyas/audio_analysis/blob/main/Greek%20Gods%20Explained%20In%2012%20Minutes.mp3?raw=true', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': True, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': True, 'content_safety_labels': {}, '

In [ ]:
# Getting the Transcription Result
result_endpoint = endpoint + "/" + response.json()["id"]
headers_auth = {
    "authorization": API_KEY,
}
transcript_response = requests.get(result_endpoint, headers=headers_auth)
print(transcript_response.json())

# While loop for requesting transcription
while response.json()['status'] != "completed":
    response = requests.get(result_endpoint, headers=headers_auth)
    time.sleep(3)

{'id': '6mrstfopff-9272-4b53-801a-f26901bff9a5', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://github.com/niveditavyas/audio_analysis/blob/main/Greek%20Gods%20Explained%20In%2012%20Minutes.mp3?raw=true', 'text': "The world began with several primordial beings. First came chaos. Then Gaia earth and finally eros love. From Gaia there came Uranus Heaven, who both created the Titans. Uranus disliked his children and locked them deep within the Earth. Gaia, sick of having her children imprisoned, made a great sickle that she gave to her son, Cronus. Cronus hid until Uranus came to lie with his mother then leapt out and sliced off his father's genitals, which fell into the sea. Cronus proceeded to release his fellow Titans from prison. And so the age of the Titans began. Cronus married the Titan, Rhea, who gave birth to most of the major gods. Cronus, scared by a prophecy that his childre

In [ ]:
JSON(transcript_response.json())

<IPython.core.display.JSON object>

In [ ]:
# Save pickle
with open('speech_data.pkl', 'wb') as f:
    pickle.dump(transcript_response.json().copy(), f)

In [ ]:
with open('speech_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
buffer = StringIO()
buffer.write(data["text"])
buffer.seek(0)

0

In [ ]:
transcript_download = pn.widgets.FileDownload(file=buffer,
                                              filename="transcript.txt",
                                              button_type='success')
transcript_download

FileDownload(button_type='success', file=<_io.StringIO o..., filename='transcript.txt', label='Download transcript.txt')

In [ ]:
audio_url = "https://github.com/niveditavyas/audio_analysis/blob/main/Greek%20Gods%20Explained%20In%2012%20Minutes.mp3"
audio_play = pn.pane.Audio(audio_url, name='Audio', time = 360)
audio_play

Audio(str, time=360)

In [ ]:
sentiment = data["sentiment_analysis_results"]

In [ ]:
sentiment_df = pd.DataFrame(sentiment)
sentiment_df

,text,start,end,sentiment,confidence,speaker
0,The world began with several primordial beings.,11850,14826,NEUTRAL,0.798996,None
1,First came chaos.,15018,16442,NEUTRAL,0.491386,None
2,Then Gaia earth and finally eros love.,16586,20158,NEUTRAL,0.792808,None
3,"From Gaia there came Uranus Heaven, who both c...",20324,24510,NEUTRAL,0.879464,None
4,Uranus disliked his children and locked them d...,24970,28710,NEGATIVE,0.875649,None
...,...,...,...,...,...,...
123,From the violent and destructive Ares to the b...,706586,717440,NEUTRAL,0.603304,None
124,"Tales pass down from each generation, showing ...",717810,727320,POSITIVE,0.890154,None
125,Thanks for watching.,729850,730982,POSITIVE,0.952933,None
126,Remember to like and subscribe.,731116,732562,NEUTRAL,0.520393,None


In [ ]:
sentiment_df_grouped = sentiment_df['sentiment'].value_counts()
sentiment_df_grouped

NEUTRAL     83
NEGATIVE    30
POSITIVE    15
Name: sentiment, dtype: int64

In [ ]:
sentiment_plot = sentiment_df_grouped.hvplot(title = "Sentences by Sentiment Category", kind="bar")
pn.Row(sentiment_plot)

Row
    [0] HoloViews(Bars, height=300, sizing_mode='fixed', width=700)

In [ ]:
positive_df = sentiment_df[sentiment_df["sentiment"] == "POSITIVE"][["text", "sentiment"]]
negative_df = sentiment_df[sentiment_df["sentiment"] == "NEGATIVE"][["text", "sentiment"]]
neutral_df = sentiment_df[sentiment_df["sentiment"] == "NEUTRAL"][["text", "sentiment"]]

sentiment_tabs = pn.Tabs(('Sentiment overview', sentiment_plot),
                       ('Positive', pn.widgets.DataFrame(positive_df, autosize_mode='fit_columns', width=700, height=300)),
                       ('Negative', pn.widgets.DataFrame(negative_df, autosize_mode='fit_columns', width=700, height=300)),
                       ('Neutral', pn.widgets.DataFrame(neutral_df, autosize_mode='fit_columns', width=700, height=300))
                        )
sentiment_tabs

Tabs
    [0] HoloViews(Bars, height=300, name='Sentiment overview', sizing_mode='fixed', width=700)
    [1] DataFrame(autosize_mode='fit_columns', height=300, sizing_mode='fixed', value=                          ..., width=700)
    [2] DataFrame(autosize_mode='fit_columns', height=300, sizing_mode='fixed', value=                          ..., width=700)
    [3] DataFrame(autosize_mode='fit_columns', height=300, sizing_mode='fixed', value=                          ..., width=700)

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
transcript = data["text"]

In [ ]:
transcript_lower = [item.lower() for item in str(transcript).split()]
transcript_lower

['the',
 'world',
 'began',
 'with',
 'several',
 'primordial',
 'beings.',
 'first',
 'came',
 'chaos.',
 'then',
 'gaia',
 'earth',
 'and',
 'finally',
 'eros',
 'love.',
 'from',
 'gaia',
 'there',
 'came',
 'uranus',
 'heaven,',
 'who',
 'both',
 'created',
 'the',
 'titans.',
 'uranus',
 'disliked',
 'his',
 'children',
 'and',
 'locked',
 'them',
 'deep',
 'within',
 'the',
 'earth.',
 'gaia,',
 'sick',
 'of',
 'having',
 'her',
 'children',
 'imprisoned,',
 'made',
 'a',
 'great',
 'sickle',
 'that',
 'she',
 'gave',
 'to',
 'her',
 'son,',
 'cronus.',
 'cronus',
 'hid',
 'until',
 'uranus',
 'came',
 'to',
 'lie',
 'with',
 'his',
 'mother',
 'then',
 'leapt',
 'out',
 'and',
 'sliced',
 'off',
 'his',
 "father's",
 'genitals,',
 'which',
 'fell',
 'into',
 'the',
 'sea.',
 'cronus',
 'proceeded',
 'to',
 'release',
 'his',
 'fellow',
 'titans',
 'from',
 'prison.',
 'and',
 'so',
 'the',
 'age',
 'of',
 'the',
 'titans',
 'began.',
 'cronus',
 'married',
 'the',
 'titan,',
 'r

In [ ]:
all_words = ' '.join(transcript_lower)
all_words

"the world began with several primordial beings. first came chaos. then gaia earth and finally eros love. from gaia there came uranus heaven, who both created the titans. uranus disliked his children and locked them deep within the earth. gaia, sick of having her children imprisoned, made a great sickle that she gave to her son, cronus. cronus hid until uranus came to lie with his mother then leapt out and sliced off his father's genitals, which fell into the sea. cronus proceeded to release his fellow titans from prison. and so the age of the titans began. cronus married the titan, rhea, who gave birth to most of the major gods. cronus, scared by a prophecy that his children would take his power, imprisoned them as soon as they were born, swallowing them whole. rhea pleaded to gaia for help, who, taking pity, hid zeus after he was born on mount ida in crete. when cronus came to eat his son, rhea fed him a stone instead, tricking the titan. gaia raised zeus until he was strong enough t

In [ ]:
# Word cloud plot
wordcloud = WordCloud(background_color='black', stopwords = stopwords, max_words = 20,
                      colormap='viridis', collocations=False).generate(all_words)

wordcloud_plot = px.imshow(wordcloud)
# Remove labels on axes
wordcloud_plot.update_xaxes(showticklabels=False)
wordcloud_plot.update_yaxes(showticklabels=False)
wordcloud_plot

In [ ]:
# Create interactive slider
class Controller(param.Parameterized):
    word_slider = param.Integer(30, bounds=(5, 50), step=5)

controller = Controller()

@pn.depends(controller.param.word_slider, watch=True)
def update_wordcloud(num_words):
    # Word cloud plot
    wordcloud = WordCloud(background_color='black', stopwords = stopwords, max_words = 20,
                          colormap='viridis', collocations=False).generate(all_words)

    wordcloud_plot = px.imshow(wordcloud)
    # Remove labels on axes
    wordcloud_plot.update_xaxes(showticklabels=False)
    wordcloud_plot.update_yaxes(showticklabels=False)
    return wordcloud_plot

In [ ]:
chapters = data["chapters"]
chapters

[{'summary': 'The world began with several primordial beings. First came chaos, then Gaia earth and finally eros love. From Gaia there came Uranus Heaven who created the Titans. And so started the reign of the gods zeus, poseidon and Hades. The underworld had several guardians and trials.',
  'gist': 'The Story of The Titans',
  'headline': 'From Gaia there came Uranus Heaven who both created the Titans',
  'start': 11850,
  'end': 417406},
 {'summary': 'Ares was the bloodthirsty and cruel god of war. Aphrodite was the goddess of beauty and love. One thing Demeter prized more than anything else was her daughter Persephone. Hermes the messenger god and patron of thieves.',
  'gist': 'Fooled by Nature',
  'headline': 'Ares was the bloodthirsty and cruel god of war',
  'start': 417598,
  'end': 699574},
 {'summary': 'Greek gods represent humanity at its best and worst. Tales pass down from each generation. Some of the most influential deities in human history. Remember to like and subscri